### Adding Realistic Substructure to the Lens Mass Model ###
This notebook adds more realistic perturbers in the form of subhalos to the mass model of the lens.
First we will import all of the relevant packages and scripts.

In [ ]:
#Initializing autoreload
%load_ext autoreload
%autoreload 2
%matplotlib inline

#Adding parent directory to working path
import os
import sys

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

#Importing packages
from astropy.visualization import AsinhStretch, ImageNormalize
import corner
import matplotlib.colors as mcolors
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import multivariate_normal
from Scripts import lens_parameters, metrics, network_predictions

### 1) Generate Sample Parameters ###
<br>
This will generate the parameters needed for generating a sample of 100 mock lens with and without substructure.

In [ ]:
#Define how many lenses will be generated in the sample
sample_num = 101

#Define how many parameters each lens has
param_num = 10

param_names = ['index','z_lens', 'gamma_md', 'theta_E_md', 'e1_md', 'e2_md', 'center_x_md', 'center_y_md', 'gamma1_md', 'gamma2_md', 'p_center_x', 'p_center_y', 'z_source', 'mag_app_source', 'R_sersic_source',
               'n_sersic_source', 'e1_source', 'e2_source', 'center_x_source', 'center_y_source', 'z_lens_light', 'mag_app_light', 'R_sersic_light', 'n_sersic_light', 'e1_light', 'e2_light', 
               'z_point_source', 'x_point_source', 'y_point_source', 'mag_app_point_source', 'sigma_sub']
#Generate the parameters to be used in the sample
param_dict = lens_parameters.perturberparameters(sample_num)
#print(param_dict)

with open('../../Data-Tables/substructure_parameters_catalog.csv', 'w+') as f:
    np.savetxt(f, param_names, fmt='%s', newline=',')
    f.write('\n')
    np.savetxt(f, param_dict, fmt='%1.15f', delimiter=',')

### 2) 100 Mock Lens Test Set With and Without Substructure ###
<br>
Here we will generate images for the network to make predictions and calculuate usefull metrics to get a better sense of how well the prediction posteriors match the known parameters.

### 2.1) Generate Images for the Network ###

In [ ]:
#Defining sample number in case the above parameter generation was not ran
sample_num = 101

#Generating images for test set without substructure
config_file = '../Configs/substructure_WoS_config.py'
image_path = '../Images_for_Network/substructure_WoS_image'
y_test, y_pred_WoS, std_pred_WoS, prec_pred_WoS = network_predictions.Predictions(config_file, image_path, sample_num=sample_num)

#Generating images for test set with substructure
config_file = '../Configs/dark_substructure_config.py'
image_path = '../Images_for_Network/substructure_WS_image'
y_test, y_pred_WS, std_pred_WS, prec_pred_WS = network_predictions.Predictions(config_file, image_path, sample_num=sample_num)

### 2.2) ###

Now we will plot all 100 lenses for each test sample and check the residuals.

In [ ]:
#Initializing array that will store the images from the image path
im_wos = []
image_path_wos = '../Images_for_Network/substructure_WoS_image/'

#Populating image array
for file in os.listdir(image_path_wos):
    if file.endswith('.npy'):
        im_wos.append(file)

#Defining details of the grid
im_wos = np.asarray(im_wos)
fig,axs = plt.subplots(10,10,figsize=(10,10))
n_cols = 10
norm_wos = ImageNormalize(np.load(image_path_wos+im_wos[1]),stretch=AsinhStretch())

for i in range(0,sample_num-1):
    axs[i//n_cols,i%n_cols].imshow(np.load(image_path_wos+im_wos[i]), norm=norm_wos)
    axs[i//n_cols,i%n_cols].set_xticks([])
    axs[i//n_cols,i%n_cols].set_yticks([])
    
plt.show()  

In [ ]:
#Initializing array that will store the images from the image path
im_ws = []
image_path_ws = '../Images_for_Network/substructure_WS_image/'

#Populating image array
for file in os.listdir(image_path_ws):
    if file.endswith('.npy'):
        im_ws.append(file)

#Defining details of the grid
im_ws = np.asarray(im_ws)
fig,axs = plt.subplots(10,10,figsize=(10,10))
n_cols = 10
norm_ws = ImageNormalize(np.load(image_path_ws+im_ws[1]),stretch=AsinhStretch())

for i in range(0,sample_num-1):
    axs[i//n_cols,i%n_cols].imshow(np.load(image_path_ws+im_ws[i]), norm=norm_ws)
    axs[i//n_cols,i%n_cols].set_xticks([])
    axs[i//n_cols,i%n_cols].set_yticks([])
    
plt.show()  

To check that the image positions are changing when we add subhalos to the main deflector we will plot the residual between the two test sets.

In [ ]:
#Setting the norm and color scale
resid_norm =mcolors.TwoSlopeNorm(vmin=-0.025,vcenter=0,vmax=0.025)

#Defining details of the grid
fig,axs = plt.subplots(10,10,figsize=(10,10))
n_cols = 10

for i in range(0,sample_num-1):
    imris = axs[i//n_cols,i%n_cols].imshow(np.load(image_path_ws+im_ws[i])-np.load(image_path_wos+im_wos[i]), norm=resid_norm,cmap='bwr')
    axs[i//n_cols,i%n_cols].set_xticks([])
    axs[i//n_cols,i%n_cols].set_yticks([])

fig.colorbar(imris, ax=axs)
plt.show()

### 2.2) Calculate Metrics ###

In [ ]:
#Generating metrics for standard deviation, accuracy, and bias
sample_num = 101
param_num = 10
mean_metrics = metrics.PerturberSampleTrunc_Substructure(sample_num, param_num, y_test, y_pred_WoS, y_pred_WS, std_pred_WoS, std_pred_WS)

#Saving metric values to a csv file
np.savetxt('../../Data-Tables/substructure_metrics_base.csv', mean_metrics, fmt="%1.2f", delimiter=",")

### 3) Interpret Output from the Network ###

In [ ]:
# Defining the learning parameters and their names
learning_params_names = [r'$\theta_\mathrm{E}$',r'$\gamma_1$',r'$\gamma_2$',r'$\gamma_\mathrm{lens}$',r'$e_1$',
								r'$e_2$',r'$x_{lens}$',r'$y_{lens}$',r'$x_{src}$',r'$y_{src}$']

#Generating corner plots for 10 lenses, this range can be user defined.
for i in range(10):
    posterior_samples_WoS = multivariate_normal(mean=y_pred_WoS[i],cov=np.linalg.inv(prec_pred_WoS[i])).rvs(size=int(5e3))
    posterior_samples_WS = multivariate_normal(mean=y_pred_WS[i],cov=np.linalg.inv(prec_pred_WS[i])).rvs(size=int(5e3))

    fig = corner.corner(posterior_samples_WoS,labels=np.asarray(learning_params_names),
                        bins=20,
                show_titles=False,plot_datapoints=False,label_kwargs=dict(fontsize=30),
                levels=[0.68,0.95],color='slategray',fill_contours=True,smooth=1.0,
                hist_kwargs={'density':True,'color':'slategray','lw':3},title_fmt='.2f',max_n_ticks=3,fig=None,
                truths=y_test[i],
                truth_color='black')
    corner.corner(posterior_samples_WS,labels=np.asarray(learning_params_names),bins=20,
                show_titles=False,plot_datapoints=False,label_kwargs=dict(fontsize=30),
                levels=[0.68,0.95],color='goldenrod',fill_contours=True,smooth=1.0,
                hist_kwargs={'density':True,'color':'goldenrod','lw':3},title_fmt='.2f',max_n_ticks=3,fig=fig)

    color = ['slategray', 'goldenrod']
    label = ['Without Substructure', 'With Substructure']
    axes = np.array(fig.axes).reshape(param_num, param_num)
    axes[0,param_num-2].legend(handles=[mlines.Line2D([], [], color=color[i], label=label[i]) for i in range(0,2)],frameon=False,
                fontsize=30,loc=10)

    axes[0,3].imshow(np.load(image_path_ws+im_ws[i]), norm=norm_ws)
    axes[0,4].imshow(np.load(image_path_wos+im_wos[i]), norm=norm_wos)
    #plt.show()
    plt.savefig('../../Images/substructure_corner_plots/corner_plot_'+str(i)+'.png')